# Лабораторная работа №3

## Криптографические протоколы на основе асимметричных криптосистем

**Задание:**

Разработать алгоритмы, реализующие криптографические протоколы (см. вариант) взаимодействия удаленных абонентов на основе асимметричных криптосистем. 

Написать программы, реализующие эти протоколы для всех участников. Значения модуля криптосистемы выбирать не менее 50 бит. Для вычислений с большими числами можно использовать специальные программы.

Для проверки чисел на простоту использовать комбинированный алгоритм на основе тестов Леманна или Рабина-Миллера
Хеширование выполнять на основе любого блочного симметричного алгоритма (например, с использованием сети Фейстеля или алгоритма из предыдущих лаб. работ) по одной из схем, данных в лекциях.

Проверить правильность выполнения протокола для малых значений параметров криптосистемы (контрольный пример).

Продемонстрировать выполнение протокола для нормальных значений параметров криптосистемы.


Задание по варианту: реализовать `протокол идентификации абонента с помощью алгоритма цифровой подписи DSA`.

Для тестирования результата сымитируем challenge-response аутентификацию: будем генерировать случайную строку заданной длины, после чего осуществим ее подпись на имитируемой стороне клиента. Далее проверим эту подпись на имитируемой стороне сервера, если она окажется корректной - аутентификация удалась

Суть алгоритма цифровой подписи [DSA](https://ru.wikipedia.org/wiki/DSA) заключается в следующем:

Пусть у нас есть базовые параметры $H(m)$ - криптографическая хеш-функция от сообщения $m$, $q$ - простое число, размерность которого в битах совпадает с размерностью в битах значений $H(m)$, $p$, такое, что $p-1$ делится на $q$ без остатка, $g$, такое, что его [мультипликативный порядок по модулю](https://ru.wikipedia.org/wiki/Мультипликативный_порядок_по_модулю) $p$ равен $q$, $x$ - секретный ключ, случайное число от 1 до $q$, $y = g^x \mod p$ - открытый ключ 

Далее мы можем подписать сообщение $m$ следующим образом:

- выбрать случайное число $k \in (0, q)$;
- вычислить $r = (g^k \mod p) \mod q$, так, чтобы $r$ не равнялось нулю. В противном случае - поменять $k$;
- вычислить $s = k^{-1}(H(m) + x \cdot r) \mod q$, так, чтобы $s$ не равнялось нулю. В противном случае - поменять $k$

Подписью сообщения будет пара ($r$, $s$)

Проверка подписи осуществляется по следующему алгоритму:

- вычисляется $w = s^{-1} \mod q$;
- вычисляется $u_1 = H(m) \cdot w \mod q$;
- вычисляется $u_2 = r \cdot w \mod q$;
- вычисляется $v = (g^{u_1} \cdot y^{u_2} \mod p) \mod q$

Если $v = r$, значит подпись верна

## Импорт необходимых библиотек

In [1]:
from core.params_generators import *
from core.signing import *

Проверим корректность и эффективность работы протокола при малых параметрах криптосистемы, а потом при приближенных к реальным 

Сгенерируем подписываемое сообщение

In [2]:
challenge = generate_challenge(1000, 9999)
challenge

'7968'

## Проверка протокола на малых параметрах

Сгенерируем параметры криптосистемы с малой битностью

In [3]:
small_params = generate_dsa_parameters(bits=66)

print(f"Закрытый ключ: {small_params[3]}")
print(f"Открытый ключ: {small_params[4]}")

Закрытый ключ: 97887929
Открытый ключ: 1516042254


Запустим процесс цифровой подписи

In [4]:
%%time
r, s = sign_message(challenge, small_params)

print(f"Сообщение было подписано: r = {r}, s = {s}")

Сообщение было подписано: r = 42791890, s = 20651287
CPU times: total: 0 ns
Wall time: 1 ms


Проверим успешность идентификации

In [5]:
%%time
signed_well = verify_signature(challenge, r, s, small_params)

print("Идентификация успешна" if signed_well else "Идентификация провалена")

Идентификация успешна
CPU times: total: 0 ns
Wall time: 1 ms


Как видно, протокол успешно отработал на малых значениях параметров криптосистемы

## Проверка протокола на реальных параметрах

Сгенерируем параметры криптосистемы с большей битностью

In [6]:
bigger_params = generate_dsa_parameters(bits=666)

print(f"Закрытый ключ: {bigger_params[3]}")
print(f"Открытый ключ: {bigger_params[4]}")

Закрытый ключ: 3314514092876822308529489145021961516349167259239800060492386510595974391908468142268222480601836242
Открытый ключ: 15804729763397345123056224991530203278253950389748557152334359535827844532408522379495167631513255958


Запустим процесс цифровой подписи

In [7]:
%%time
r, s = sign_message(challenge, bigger_params)

print(f"Сообщение было подписано: r = {r}, s = {s}")

Сообщение было подписано: r = 2513672138068741840393466388624152781273501510052025560877167487594775223593590233912584286382273517, s = 790416902515980663293363423867731418707009296050333833497332271362210327718173972660224536750860500
CPU times: total: 0 ns
Wall time: 999 µs


Проверим успешность идентификации

In [8]:
%%time
signed_well = verify_signature(challenge, r, s, bigger_params)

print("Идентификация успешна" if signed_well else "Идентификация провалена")

Идентификация успешна
CPU times: total: 0 ns
Wall time: 1.02 ms


Как видно, протокол успешно отработал и на реальных значениях параметров криптосистемы, причем скорость работы алгоритма при увеличении битности параметров более в чем 10 раз практически не изменилась